In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date
import plotly.express as px
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss, ccf
from math import ceil
from os import listdir
from os.path import isfile, join
import holidays

pd.set_option('display.max_rows', 300)

## Helper Functions

In [16]:
def clean_en_data(df, df_name):
    sampfreqsec_2_str = {
        900:"15min",
        1800:"30min",
        3600:"1h",
    }
        
    df = df.copy()
    
    n_dup = df.duplicated().sum()
        
    if n_dup > 0:
        df = df.drop_duplicates()
    
    n_df_pre_reindex = len(df)
    sampfreqsec = df["datetime"].diff()[1:].describe()['50%'].seconds
    new_index = pd.date_range(df['datetime'].min(), df['datetime'].max(),freq=sampfreqsec_2_str[sampfreqsec])
    n_new_index = len(new_index)   
    df = df.set_index("datetime")
    
    if n_df_pre_reindex != n_new_index:
        # Missing data entries, need to reindex
        df = df.reindex(new_index)

    if sampfreqsec_2_str[sampfreqsec] != "1h":
        if df_name == "generation_forecast":
            df = df.resample('1h',closed='right', label='right').mean()
        else:
            df = df.resample('1h',closed='right', label='right').sum()
        
    return df

def prepare_en_data(df_names):
    
    print("Preparing Energy Data")
    
    dfs = {}

    for df_name in df_names:
        dfs[df_name] = pd.read_csv(f".\\data\\{df_name}_NL.csv") 
        dfs[df_name]["datetime"] = pd.to_datetime(dfs[df_name]["datetime"], utc=True).dt.tz_localize(None)
        dfs[df_name] = dfs[df_name].sort_values('datetime')
        dfs[df_name] = clean_en_data(dfs[df_name], df_name)

    # remove Hydro Run-of-river and poundage from generation
    dfs["generation"] = dfs["generation"].drop('Hydro Run-of-river and poundage', axis=1)
    # Add generation prefix to generation adn wind_solar_forecast columns
    dfs["generation"] = dfs["generation"].add_prefix("generation_")
    dfs["wind_solar_forecast"] = dfs["wind_solar_forecast"].add_prefix("generation_forecast_")
    
    check = 0
    for i, df_name in enumerate(df_names):
        temp_n = len(dfs[df_name])
        if i == 0:
            df = dfs[df_name].copy()
        elif i > 0 and temp_n != check:
            print(f"Different number of rows in {df_name}({temp_n}) vs {df_names[i-1]}({check})")
            return pd.DataFrame()
        else:
            df = pd.merge_asof(df,  dfs[df_name], left_index=True, right_index=True)
            
        check = temp_n
        
    # set outliers to null and interpolate    
    outlier_ind = df[df["generation_Waste"] < 0.0].index
    df.loc[outlier_ind, "generation_Waste"] = np.nan
    df.interpolate(method='linear', limit_direction='forward', inplace=True)
    
    # calculate total generation
    generation_columns = [col for col in df.columns if ("generation_" in col) and ("missing_" not in col)]
    df["generation_total"] = df[generation_columns].sum(axis=1)
    
    # Calculate residual load and drop actual load
    df["generation_renewable"] = df[["generation_Solar", "generation_Wind Offshore", "generation_Wind Onshore"]].sum(axis=1)
    df["residual_load"] = df["Actual Load"].sub(df["generation_renewable"])
    df = df.drop(["generation_renewable", "Actual Load"], axis=1)
    
    # Combine generation_Wind variables and drop actual load
    df["generation_Wind"] = df[["generation_Wind Offshore", "generation_Wind Onshore"]].sum(axis=1)
    df = df.drop(["generation_Wind Offshore", "generation_Wind Onshore"], axis=1)
    
    # Combine generation_forecast_Wind variables and drop actual load
    df["generation_forecast_Wind"] = df[["generation_forecast_Wind Onshore", "generation_forecast_Wind Offshore"]].sum(axis=1)
    df = df.drop(["generation_forecast_Wind Onshore", "generation_forecast_Wind Offshore"], axis=1)
    
    return df

def get_elec_consumption_by_city(cities):
    path = ".\\data\\Electricity"
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f)) and ('2020' in f)]

    df_en = []

    for f in onlyfiles:
        df_en.append(pd.read_csv(f))

    df_en = pd.concat(df_en, ignore_index=True)
    df_en['city'] = df_en['city'].str.lower()
    df_en = df_en.groupby('city')['annual_consume'].sum().sort_values(ascending=False)
    df_en = df_en.rename(index={"'s-gravenhage":"the hague"})
    df_en = df_en[cities]
    dict_en = df_en.to_dict()
    
    return dict_en

def create_weighted_variables(df, corr_features, cities):
    
    dict_weights = get_elec_consumption_by_city(cities)
    cols_2_remove = []

    for feature in corr_features:
        temp_weighted = 0.0
        total_weight = 0.0
        for city in cities:
            if f"{feature}_{city}" in df.columns:
                cols_2_remove.append(f"{feature}_{city}")
                weight = dict_weights[city]
                total_weight += weight
                temp_weighted += weight * df[f"{feature}_{city}"]
        temp_weighted = temp_weighted / total_weight
        df[f"{feature}_weighted"] = temp_weighted
        
    df = df.drop(cols_2_remove, axis=1)
    
    return df
    

def prepare_weather_data(min_datetime, max_datetime):
    
    print("Preparing Weather Data")
    
    # Load data
    df = pd.read_csv(".\\data\\weather_data_NL.csv")
    df = df.rename(columns={"dt":"datetime"})
    df["datetime"] = pd.to_datetime(df["datetime"], unit='s')
    df["city_name"] = df["city_name"].str.lower()
    df = df[(df["datetime"] >= min_datetime) & (df["datetime"] <= max_datetime)].copy()
    df = df.sort_values(['datetime','city_name'])
        
    # remove irrevalant columns
    cols_2_remove = ['dt_iso', 'timezone', 'lat', 'lon', 'weather_icon', 'weather_id', 'weather_main', 'weather_description',
                     'visibility', 'wind_deg', 'clouds_all', 'feels_like', 'temp_min', 'temp_max']
    
    df = df.drop(cols_2_remove, axis=1)
    # remove features with majority null values
    df_pct_null = df.isna().sum() / len(df)
    cols_2_remove = list(df_pct_null[df_pct_null > 0.5].index)
    df = df.drop(cols_2_remove, axis=1)
    # remove duplicates
    df = df.drop_duplicates()
    count_by_city = df.groupby('city_name').size()
    
    # check that there are the same number of records per city
    for i in range(len(count_by_city)):
        if i > 0 and count_by_city.iloc[i] != count_by_city.iloc[0]:
            city1, count1 = count_by_city.index[i-1], count_by_city.iloc[i-1]
            city2, count2 = count_by_city.index[i], count_by_city.iloc[i]
            print(f"Different number of records for {city1}({count1}) vs {city2}({count2})")
            return pd.DataFrame()
    
    df = df.set_index("datetime")
    
    # set outliers to null
    outlier_ind = []
    outlier_ind.append((df['city_name'] == "utrecht") & (df['wind_speed'] > 21) & (df.index >= '2021-01-24') & (df.index <= '2021-01-26'))
    outlier_ind.append((df['city_name'] == "utrecht") & (df['wind_speed'] > 21) & (df.index >= '2021-03-10') & (df.index <= '2021-03-20'))
    outlier_ind.append((df['city_name'] == "eindhoven") & (df['wind_speed'] > 10) & (df.index >= '2022-11-16') & (df.index <= '2022-11-25'))
    outlier_ind.append((df['city_name'] == "eindhoven") & (df['wind_speed'] > 15) & (df.index >= '2023-01-13') & (df.index <= '2023-01-15'))
    outlier_ind.append((df['humidity'] < 18))   
    outlier_cols = ["wind_speed", "wind_speed", "wind_speed", "wind_speed", ["dew_point", "humidity"]]
 
    for ind, cols in zip(outlier_ind, outlier_cols):
        df.loc[ind, cols] = np.nan
        
    cities = df["city_name"].unique()

    # create seperate column per feature and city                  
    for i, city in enumerate(cities):
        temp_df = df[df["city_name"] == city].copy()
        temp_df = temp_df.drop("city_name", axis=1)
        temp_df = temp_df.sort_index()
        temp_df = temp_df.add_suffix(f'_{city}')
        if i == 0:
            df_wide = temp_df
        else:
            df_wide = pd.merge(df_wide, temp_df, left_index=True, how='left', right_index=True)

    df = df_wide.copy()
    
    del df_wide
                    
    # remove features with majority null values
    df_pct_null = df.isna().sum() / len(df)
    cols_2_remove = list(df_pct_null[df_pct_null > 0.5].index)
    df = df.drop(cols_2_remove, axis=1)
                       
    # interpolate remaining features          
    df.interpolate(method='linear', limit_direction='forward', inplace=True)    
    
#     # create temperature ranges per city and remove feels_like*, temp_min*, temp_max*
#     fl_features = [f"feels_like_{city}" for city in cities if f"feels_like_{city}" in df.columns]
#     tmin_features = [f"temp_min_{city}" for city in cities if f"temp_min_{city}" in df.columns]
#     tmax_features = [f"temp_max_{city}" for city in cities if f"temp_max_{city}" in df.columns]
#     df_temp_range = df[tmax_features].sub(df[tmin_features].rename(columns=dict(zip(tmin_features, tmax_features))))
#     df_temp_range.columns = [col.replace("temp_max", "temp_rng") for col in df_temp_range.columns]
#     df = df.drop(fl_features + tmin_features + tmax_features, axis=1)  
#     df = pd.merge(df, df_temp_range, how='left', left_index=True, right_index=True)
    
    # Create weighted variable from highly correlated variables per city 
    corr_features = ["dew_point", "temp", "pressure", "humidity", "wind_speed"]
    df =  create_weighted_variables(df, corr_features, cities)
    
    return df

def prepare_unav_gen_data(df_names):
    
    print("Preparing Unavailable Generation Data")
    
    for i, df_name in enumerate(df_names):
        temp_df = pd.read_csv( f".\\data\\unavailability_of_generation_units_NL_{df_name}.csv") 
        temp_df["datetime"] = pd.to_datetime(temp_df["datetime"], utc=True).dt.tz_localize(None)
        temp_df = temp_df.sort_values('datetime').set_index('datetime')
        if i == 0:
            df = temp_df.copy()
        else:
            df = pd.merge(df, temp_df, left_index=True, how='left', right_index=True)
            
    return df.fillna(0)

def generate_calendar_features(df):
    df = df.copy()
    print("Adding Calendar Features")
    df["hour_of_day"] = df["datetime"].dt.hour
    df["day_of_week"] = df["datetime"].dt.dayofweek
    df["weekend"] = np.where((df["day_of_week"] == 5) | (df["day_of_week"] == 6), 1, 0)
    df["month"] = df["datetime"].dt.month
    # add variables for holidays
    nl_holidays = holidays.NL() 
    df["holiday"] = df["datetime"].dt.date.apply(lambda x: nl_holidays.get(x)).values
    df["is_holiday"] = np.where(df["holiday"].isna(), 0, 1)
    df = pd.get_dummies(df, columns=["holiday"], dummy_na=False)
    df = df.drop(["holiday_Bevrijdingsdag", "holiday_Koningsdag"], axis=1)
    holiday_cols = [col for col in df.columns if "holiday_" in col]
    for col in holiday_cols:
        df[col] = np.where(df[col], 1, 0)
    
    return df

def generate_RLI_features(df, horizons):
    df = df.copy()
    print("Adding Relative Load Indicator Features")
    for h in horizons:
        temp_residual_load_rs = df["residual_load"].rolling(window=h,min_periods=h).sum().shift(1)
        df[f"RLI_{h}"] = 100 * df["residual_load"].div(temp_residual_load_rs)
        del temp_residual_load_rs
    
    return df

def generate_lagged_features(df, features, lags):
    lagged_data = []
    for feature in features:
        for lag in lags:
            # df[f'{feature}_lag_{lag}'] = df[feature].shift(lag)
            lagged_data.append(df[feature].shift(lag))
            lagged_data[-1].name = f'{feature}_lag_{lag}'

    df = pd.concat([df] + lagged_data, axis=1)

    return df.copy()

def feature_generation(df):
    df = df.reset_index()
    df = generate_calendar_features(df)
    df = generate_RLI_features(df, [24, 48, 120, 168])
    lags = [120]
#     lags = [24 * i for i in range(1,8)]
    print("Adding Lagged day_ahead_prices")
    df = generate_lagged_features(df, ["day_ahead_prices"], lags)
    
    features = [
        'generation_Biomass',
        'generation_Fossil Gas', 'generation_Fossil Hard coal',
        'generation_Nuclear', 'generation_Other', 'generation_Solar',
        'generation_Waste', 'crossborder_flow_net', 'imports',
        'Forecasted Load', 'generation_forecast', 'generation_forecast_Solar',
        'generation_total', 'residual_load', 'generation_Wind',
        'generation_forecast_Wind', 
        'dew_point_weighted', 'temp_weighted', 'pressure_weighted',
        'humidity_weighted', 'wind_speed_weighted',
        'generation_fossil_gas_missing_qty',
        'generation_fossil_hard_coal_missing_qty',
        'generation_nuclear_missing_qty'
    ]
    lags = [24, 48, 72, 96, 120]
    print("Adding Lagged covariates")
    df = generate_lagged_features(df, features, lags)
    # Negatively shift prices by 48hours so we forecast 48 hours into the future
    print("Adding leading day_ahead_returns as target")
    df['day_ahead_prices_lead_48'] = df['day_ahead_prices'].shift(-48)
    df = df.dropna()
    return df

In [17]:
en_names = [    
    "day_ahead_prices",
    "load",
    "generation",
    "crossborder_flow_net",
    "imports",
    "load_forecast",
    "generation_forecast",
    "wind_solar_forecast",
]

gen_names = [    
    "fossil_gas",
    "fossil_hard_coal",
    "nuclear"
]

dfs = []

print("#" * 20)
print("Data Preparation")
print("#" * 20)
print("")
dfs.append(prepare_en_data(en_names))
dfs.append(prepare_weather_data(dfs[0].index.min(), dfs[0].index.max()))
dfs.append(prepare_unav_gen_data(gen_names))

print("Merging Data")
for i, temp_df in enumerate(dfs):
    if i == 0:
        df = temp_df.copy()
    else:
        df = pd.merge(df, temp_df, left_index=True, how='left', right_index=True)

print("\nData Quality")
df_nulls_zeros = pd.DataFrame()
df_nulls_zeros["pct_nulls"] = 100 * (df.isna().sum() / len(df)).to_frame().round(1)
df_nulls_zeros["pct_zeros"] = 100 * ((df == 0).sum() / len(df)).to_frame().round(1)
print(df_nulls_zeros)

print("#" * 20)
print("Feature Generation")
print("#" * 20)
print("")
df = feature_generation(df)

df.to_csv(".\\data\\prepped_data_NL.csv", index=False)

####################
Data Preparation
####################

Preparing Energy Data
Preparing Weather Data
Preparing Unavailable Generation Data
Merging Data

Data Quality
                                         pct_nulls  pct_zeros
day_ahead_prices                               0.0        0.0
generation_Biomass                             0.0       10.0
generation_Fossil Gas                          0.0        0.0
generation_Fossil Hard coal                    0.0        0.0
generation_Nuclear                             0.0        0.0
generation_Other                               0.0        0.0
generation_Solar                               0.0       10.0
generation_Waste                               0.0        0.0
crossborder_flow_net                           0.0        0.0
imports                                        0.0        0.0
Forecasted Load                                0.0        0.0
generation_forecast                            0.0        0.0
generation_forecast_Sola

In [18]:
df.iloc[0]

datetime                                           2020-01-08 00:00:00
day_ahead_prices                                                 25.77
generation_Biomass                                                38.0
generation_Fossil Gas                                          12153.0
generation_Fossil Hard coal                                     3548.0
generation_Nuclear                                              1940.0
generation_Other                                               12485.0
generation_Solar                                                   0.0
generation_Waste                                                1700.0
crossborder_flow_net                                            -570.0
imports                                                         2242.0
Forecasted Load                                                38452.0
generation_forecast                                             7616.0
generation_forecast_Solar                                          0.0
genera